In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/datasets/mvp_data.csv', parse_dates=True, index_col=0)
tickers = ['EUR', 'GOLD', 'Bitcoin', 'Apple', 'Exxon', 'VISA', 'Oil']
df.columns = tickers
df_w = df.fillna(method='ffill').resample('W').ffill()
df_w_pct = df_w.pct_change().iloc[1:]
df_w_pct.head()

,EUR,GOLD,Bitcoin,Apple,Exxon,VISA,Oil
Date,,,,,,,
2020-01-12,-0.005400,0.005358,0.111870,0.043372,-0.016920,0.021994,-0.063600
2020-01-19,0.002562,0.000835,0.093367,0.027068,-0.008389,0.056407,-0.008469
2020-01-26,-0.007573,0.007891,-0.054161,-0.001318,-0.032672,0.001466,-0.074308
2020-02-02,-0.002360,0.007511,0.107170,-0.027646,-0.063329,-0.029415,-0.048533
2020-02-09,-0.004185,-0.009034,0.047635,0.036443,-0.010464,0.018948,-0.024050


In [3]:
train_df = df_w_pct[:'2021-12-31'] # 2020-2021
test_df = df_w_pct['2022-01-01':] # 2022

In [4]:
num_assets = len(tickers)

mean_returns = train_df.mean()
cov_matrix = train_df.cov()


## СИМУЛЯЦИЯ

In [5]:
#массив из нулей
num_iter = 1000
simulations = np.zeros((4 + len(tickers)- 1, num_iter))

for i in range(num_iter):
        # случайные веса + нормализация, чтобы сумма 1
        weights = np.array(np.random.random(num_assets))
        weights /= np.sum(weights)
        
        # доходность и стандартное отклонение
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))
        simulations[0,i] = portfolio_return
        simulations[1,i] = portfolio_std_dev
        
        # кэф Шарпа
        simulations[2,i] = simulations[0,i] / simulations[1,i]
        
        # dtcf
        for j in range(len(weights)):
                simulations[j+3,i] = weights[j]

# результат
df_sim = pd.DataFrame(simulations.T, 
                         columns=['RETURN','stdev','Sharpe', tickers[0], tickers[1], tickers[2], tickers[3], tickers[4], tickers[5], tickers[6]])

In [6]:
# максимальный Шарп
max_sharpe = df_sim.iloc[df_sim['Sharpe'].idxmax()]

# минимальное ст. отклонение
min_std = df_sim.iloc[df_sim['stdev'].idxmin()]

# максимальная прибыль
max_profit = df_sim.iloc[df_sim['RETURN'].idxmax()]

print ("Портфель с макс. Шарпом:", max_sharpe, sep='\n')

Портфель с макс. Шарпом:
RETURN     0.010494
stdev      0.039072
Sharpe     0.268585
EUR        0.007260
GOLD       0.318395
Bitcoin    0.272878
Apple      0.296706
Exxon      0.022542
VISA       0.056314
Oil        0.025904
Name: 880, dtype: float64


In [7]:
print ("Портфель с мин. риском:", min_std, sep='\n')

Портфель с мин. риском:
RETURN     0.003223
stdev      0.022334
Sharpe     0.144327
EUR        0.301951
GOLD       0.221720
Bitcoin    0.010542
Apple      0.195620
Exxon      0.048282
VISA       0.214250
Oil        0.007634
Name: 268, dtype: float64


In [8]:
print ("Портфель с макс. прибылью:", max_profit, sep='\n') # не всегда совпадает с макс. шарпом

Портфель с макс. прибылью:
RETURN     0.014166
stdev      0.058851
Sharpe     0.240714
EUR        0.014223
GOLD       0.110647
Bitcoin    0.472784
Apple      0.163474
Exxon      0.147978
VISA       0.006129
Oil        0.084766
Name: 987, dtype: float64


In [9]:
# сохраняем модель
# df_sim.to_pickle('model_monte_carlo.pkl')

In [16]:
round(max_sharpe.GOLD * 10000, 2)

3183.95

In [12]:
max_profit

RETURN     0.014166
stdev      0.058851
Sharpe     0.240714
EUR        0.014223
GOLD       0.110647
Bitcoin    0.472784
Apple      0.163474
Exxon      0.147978
VISA       0.006129
Oil        0.084766
Name: 987, dtype: float64

In [18]:
# проверки по модели
model = pd.read_pickle('../models/model_monte_carlo.pkl')
portfolio = model.iloc[model['Sharpe'].idxmax()]
depo = 1000
[round(portfolio.EUR * depo, 2), round(portfolio.GOLD * depo, 2), round(portfolio.Bitcoin * depo, 2)]

[130.11, 140.74, 327.12]